Universidad del Valle de Guatemala
<br>
Inteligencia Artificial
<br>
Andrea Argüello 17801
<br>
<center><b><font size="6">Laboratorio 2: Redes Neuronales</font></b></center>

# 1. Modelo
## 1.1 Modelo #1

In [1]:
import numpy as np
import pandas as pd
import neural_networks as nn
from scipy.optimize import minimize
from collections import Counter, OrderedDict, defaultdict

In [2]:
#df = pd.read_csv(r'./datasets/csv/fashion-mnist_train.csv')
test = pd.read_csv(r'./datasets/csv/fashion-mnist_test.csv')

#shuffled = df.sample(frac=1)

#train, validate = np.split(shuffled, [50000])
#train.to_csv(path_or_buf='./datasets/csv/fashion-mnist_train_50k.csv',index=False)
#validate.to_csv(path_or_buf='./fashion-mnist_validate.csv',index=False)

train = pd.read_csv(r'./datasets/csv/fashion-mnist_train_50k.csv')
validate = pd.read_csv(r'./datasets/csv/fashion-mnist_validate.csv')

NORMALIZE = 1000.0
train_X = train[train.columns[~train.columns.isin(['label'])]].to_numpy() / NORMALIZE
train_y = train[['label']].to_numpy()
validate_X = validate[validate.columns[~validate.columns.isin(['label'])]].to_numpy() / NORMALIZE
validate_y = validate[['label']].to_numpy()
test_X = test[test.columns[~test.columns.isin(['label'])]].to_numpy() / NORMALIZE
test_y = test[['label']].to_numpy()

train_Y = (train_y == np.asarray(range(10))).astype(int)
validate_Y = (validate_y == np.asarray(range(10))).astype(int)
test_Y = (test_y == np.asarray(range(10))).astype(int)

In [3]:
# A setear variables
NETWORK_ARCH = np.array([
    784,
    90, # ~(784*10)**0.5
    10
    ])
theta_shapes = np.hstack((
    NETWORK_ARCH[1:].reshape(len(NETWORK_ARCH)-1,1),
    (NETWORK_ARCH[:-1]+1).reshape(len(NETWORK_ARCH)-1,1)
))

In [4]:
# Otras variables
#flat_thetas = nn.flatten_list_of_arrays([
#    np.random.rand(*theta_shape) for theta_shape in theta_shapes
#])

In [5]:
flat_thetas = np.load('flat_thetas3.npy')

### 1.1.1 Optimización con scipy.optimize.minimize
Esto se realizó en el archivo jupyter.py por aparte, ya que en jupyter se quedó en caché un error que persistió sin importar cuantas veces reiniciase el kernel.

In [6]:
result = np.load('result.npy') #este es result.x
#result = result.reshape(len(result),1)

### 1.1.2 Accuracy con el train set

In [7]:
new_thetas = nn.inflate_matrices(result, theta_shapes)

In [8]:
new_a_train = nn.feed_forward(new_thetas, train_X)

In [9]:
# Se obtienen los indices del porcentaje mas alto
def accuracy(predictions, y):
    pred = np.argmax(predictions[-1], axis = 1).reshape(len(predictions[-1]),1)
    correct = ((pred==y) * 1).sum()
        
    print("Success: ", correct, " out of ", len(predictions[-1]), ", i.e. ", correct * 100/len(predictions[-1]))

In [10]:
accuracy(new_a_train, train_y)

Success:  44172  out of  50000 , i.e.  88.344


### 1.1.3 Accuracy del cross validation set

In [12]:
new_a_validate = nn.feed_forward(new_thetas, validate_X)
accuracy(new_a_validate, validate_y)

Success:  8503  out of  10000 , i.e.  85.03


### 1.1.4 Análisis previo a Test set
Veamos cuáles son las imágenes que más le cuesta identificar...

#### 1.1.4.1 Training set

In [13]:
labels = ["T-shirt/top", "Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

In [14]:
def freq_table(predictions, y):
    assignation = []
    pred = np.argmax(predictions[-1], axis = 1).reshape(len(predictions[-1]),1).tolist()
    y=y.tolist()
    for i in range(len(pred)):
        assignation.append(tuple([labels[y[i][0]], labels[pred[i][0]]]))
    return Counter(map(tuple,tuple(assignation)))

def totals_table(assignation_freq_table):
    mapping = defaultdict(list) #You do not need a defaultdict per se, i just find them more graceful when you do not have a certain key.
    for k in OrderedDict(sorted(assignation_freq_table.items())):
        mapping[k[0]].append(k)
    totals = defaultdict(list)
    for k in mapping:
        all_k = 0
        for v in mapping[k]:
            all_k+=assignation_freq_table[v]
        totals[k].append(all_k)
    return totals

def freq_and_totals(predictions,y):
    return freq_table(predictions,y), totals_table(freq_table(predictions,y)) 

In [15]:
assignation_freq, totals = freq_and_totals(new_a_train,train_y)

In [16]:
for k,v in OrderedDict(sorted(assignation_freq.items())):
    print("Expected ",k,", got", v, ": ", assignation_freq[k,v], "out of", totals[k][0], "i.e.", round(assignation_freq[k,v]*100/totals[k][0],2),"%")

Expected  Ankle boot , got Ankle boot :  4896 out of 5040 i.e. 97.14 %
Expected  Ankle boot , got Bag :  2 out of 5040 i.e. 0.04 %
Expected  Ankle boot , got Sandal :  26 out of 5040 i.e. 0.52 %
Expected  Ankle boot , got Sneaker :  116 out of 5040 i.e. 2.3 %
Expected  Bag , got Ankle boot :  3 out of 4983 i.e. 0.06 %
Expected  Bag , got Bag :  4847 out of 4983 i.e. 97.27 %
Expected  Bag , got Coat :  16 out of 4983 i.e. 0.32 %
Expected  Bag , got Dress :  16 out of 4983 i.e. 0.32 %
Expected  Bag , got Pullover :  18 out of 4983 i.e. 0.36 %
Expected  Bag , got Sandal :  2 out of 4983 i.e. 0.04 %
Expected  Bag , got Shirt :  53 out of 4983 i.e. 1.06 %
Expected  Bag , got Sneaker :  4 out of 4983 i.e. 0.08 %
Expected  Bag , got T-shirt/top :  22 out of 4983 i.e. 0.44 %
Expected  Bag , got Trouser :  2 out of 4983 i.e. 0.04 %
Expected  Coat , got Bag :  1 out of 4972 i.e. 0.02 %
Expected  Coat , got Coat :  4016 out of 4972 i.e. 80.77 %
Expected  Coat , got Dress :  140 out of 4972 i.e. 2

##### Esta es mucha información, así que, descartemos los casos que sean muy bajos, i.e., menores al 5%, solo para tener una mejor idea.

In [17]:
for k,v in OrderedDict(sorted(assignation_freq.items())):
    if(round(assignation_freq[k,v]*100/totals[k][0],2)>=5):
        print("Expected ",k,", got", v, ": ", assignation_freq[k,v], "out of", totals[k][0], "i.e.", round(assignation_freq[k,v]*100/totals[k][0],2),"%")

Expected  Ankle boot , got Ankle boot :  4896 out of 5040 i.e. 97.14 %
Expected  Bag , got Bag :  4847 out of 4983 i.e. 97.27 %
Expected  Coat , got Coat :  4016 out of 4972 i.e. 80.77 %
Expected  Coat , got Pullover :  443 out of 4972 i.e. 8.91 %
Expected  Coat , got Shirt :  345 out of 4972 i.e. 6.94 %
Expected  Dress , got Dress :  4372 out of 4999 i.e. 87.46 %
Expected  Pullover , got Coat :  550 out of 4995 i.e. 11.01 %
Expected  Pullover , got Pullover :  3974 out of 4995 i.e. 79.56 %
Expected  Pullover , got Shirt :  342 out of 4995 i.e. 6.85 %
Expected  Sandal , got Sandal :  4940 out of 5015 i.e. 98.5 %
Expected  Shirt , got Coat :  444 out of 5010 i.e. 8.86 %
Expected  Shirt , got Pullover :  541 out of 5010 i.e. 10.8 %
Expected  Shirt , got Shirt :  3151 out of 5010 i.e. 62.89 %
Expected  Shirt , got T-shirt/top :  696 out of 5010 i.e. 13.89 %
Expected  Sneaker , got Sneaker :  4855 out of 5006 i.e. 96.98 %
Expected  T-shirt/top , got Shirt :  374 out of 4996 i.e. 7.49 %
Exp

<b> NOTA: </b> 
    <ul> 
        <li> Se puede observar que en el training set, el accuracy más bajo es el de las "Shirts", siendo la clasificación de estas correcto el 62.89% de los casos. Se confunde usualmente con "T-shirt/top" y "Pullover" el 13.89% y 10.8%, respectivamente. También suele confundirse con "Coat," pero esto ocurre en un 8.86% de los casos.
        <li> Le sigue "Pullover", con 79.56% de asignaciones correctas. Se clasifica erróneamente como "Coat" un 11.01% de las veces, y con "Shirt" en un 6.94%.
        <li> Coat es el tercero más bajo, con 80.77%. También suele confundirse con "Pullover" y "Shirt".
        <li> Cabe destacar además que ninguno de los elementos tuvo un accuracy del 100%. El más alto due el de "Sandal" con 98.5% de accuracy. 
        <li> Si bien "Bag" con 97.27% de accuracy es el segundo más alto, este también se asignó incorrectamente a una gran cantidad de elementos, sin ningún parecido en común. Esto podría deberse a la resolución de la imagen.
    </ul>

#### 1.1.4.2 Cross validation set

In [18]:
assignation_freq_cv, totals_cv = freq_and_totals(new_a_validate,validate_y)
for k,v in OrderedDict(sorted(assignation_freq_cv.items())):
    print("Expected ",k,", got", v, ": ", assignation_freq_cv[k,v], "out of", totals_cv[k][0], "i.e.", round(assignation_freq_cv[k,v]*100/totals_cv[k][0],2),"%")
print("\n------------------Sobre 5%-------------------")
for k,v in OrderedDict(sorted(assignation_freq_cv.items())):
    if(round(assignation_freq_cv[k,v]*100/totals_cv[k][0],2)>=5):
        print("Expected ",k,", got", v, ": ", assignation_freq_cv[k,v], "out of", totals_cv[k][0], "i.e.", round(assignation_freq_cv[k,v]*100/totals_cv[k][0],2),"%")

Expected  Ankle boot , got Ankle boot :  907 out of 960 i.e. 94.48 %
Expected  Ankle boot , got Bag :  1 out of 960 i.e. 0.1 %
Expected  Ankle boot , got Pullover :  1 out of 960 i.e. 0.1 %
Expected  Ankle boot , got Sandal :  18 out of 960 i.e. 1.88 %
Expected  Ankle boot , got Sneaker :  33 out of 960 i.e. 3.44 %
Expected  Bag , got Bag :  949 out of 1017 i.e. 93.31 %
Expected  Bag , got Coat :  5 out of 1017 i.e. 0.49 %
Expected  Bag , got Dress :  5 out of 1017 i.e. 0.49 %
Expected  Bag , got Pullover :  12 out of 1017 i.e. 1.18 %
Expected  Bag , got Sandal :  2 out of 1017 i.e. 0.2 %
Expected  Bag , got Shirt :  26 out of 1017 i.e. 2.56 %
Expected  Bag , got Sneaker :  9 out of 1017 i.e. 0.88 %
Expected  Bag , got T-shirt/top :  8 out of 1017 i.e. 0.79 %
Expected  Bag , got Trouser :  1 out of 1017 i.e. 0.1 %
Expected  Coat , got Bag :  5 out of 1028 i.e. 0.49 %
Expected  Coat , got Coat :  814 out of 1028 i.e. 79.18 %
Expected  Coat , got Dress :  34 out of 1028 i.e. 3.31 %
Expec

<b> NOTA: </b> 
    <ul> 
        <li> Se mantiene que "Shirt", "Pullover" y "Coat" son los primers tres más bajos, respectivamente. Sin embargo, puede notarse que los porcentajes erróneos, especialmente clasificar errónamente una "Shirt" como un "Coat", han aumentado. Sin embargo, para "Coat," esta diferencia es menor.
        <li> Nuevamente, ninguno de los elementos tuvo un accuracy del 100%, pero esta vez el más alto fue el de "Trouser," cuyo porcentaje de accuracy es bastante parecido al del training set. "Sandal" disminuyó significativamente, y hay casos en los que se clasificó erróneamente como "Bag" y "Dress," al igual que aumentó el porcentaje de veces que se clasificó como "Ankle boot" o "Sneaker."
        <li> Si bien "Bag" disminuyó en accuracy, se puede ver que mientras en el training set se clasificó al menos una vez en cada uno de los labels erróneos, en el cross validation set no se ha clasificado en ninguna instanca como "Ankle boot".
    </ul>

Si bien el accuracy en general tanto para el training y el cross validation sets, debido a estas observaciones, vale la pena preguntarse si un modelo con más neuronas aumentaría el accuracy. Según las observaciones, podría ser seguro asumir que hasta el momento no se presenta overfitting, si no tal vez algún tipo de bias, ya que todas las categorias presentan incluso en el training set al menos 3 categorizaciones erróneas, algunas incluso que no parecen hacer sentido (como todo lo que se clasifica erróneamente como "Bag", o vestimentas que se confundieron con alguno de los tipos de zapato)

## 1.2 Modelo #2
Este, nuevamente por problemas de cache de jupyter con respecto a scipy, puede encontrarse en el archivo jupyter2.py. Esta tenía poco menos de 1.5x la cantidad de neuronas en la capa escondida de la anterior.

In [19]:
new_thetas_2 = np.load('result_model_2.npy')
NETWORK_ARCH_2 = np.array([
    784,
    round((3/2)*(784*10)**0.5), # Miremos que pasa si aumentamos la cantidad de neuronas
    10
])
theta_shapes_2 = np.hstack((
    NETWORK_ARCH_2[1:].reshape(len(NETWORK_ARCH_2)-1,1),
    (NETWORK_ARCH_2[:-1]+1).reshape(len(NETWORK_ARCH_2)-1,1)
))
print(NETWORK_ARCH_2[1])

133


In [20]:
train_model_2 = nn.feed_forward(nn.inflate_matrices(new_thetas_2, theta_shapes_2), train_X)
validate_model_2 = nn.feed_forward(nn.inflate_matrices(new_thetas_2, theta_shapes_2), validate_X)

accuracy(train_model_2,train_y)
accuracy(validate_model_2,validate_y)

Success:  50000  out of  50000 , i.e.  100.0
Success:  8714  out of  10000 , i.e.  87.14


Dado que se obtuvo un resultado para train del 100%, pero de 87% para el cross validation set, descartamos este modelo, ya que parece ser un caso de overfitting.

# 2. Test Set
Utilizando el primer modelo que teníamos, evaluemos el resultado con el test set (no vale la pena evaluar el segundo modelo, ya que, como se mencionó anteriormente, al obtener un 100% de accuracy para el training set, se sospecha que este está overfitteado):

In [29]:
a_test_1 = nn.feed_forward(new_thetas, test_X)
print('****************Modelo 1: 1 hidden layer de 90 neuronas****************')
accuracy(a_test_1,test_y)

print('\n!!!!!!!!!!!!!!!FREQUENCIES!!!!!!!!!!!!!!!')
assignation_freq_t1, totals_t1 = freq_and_totals(a_test_1,test_y)
for k,v in OrderedDict(sorted(assignation_freq_t1.items())):
    print("Expected ",k,", got", v, ": ", assignation_freq_t1[k,v], "out of", totals_t1[k][0], "i.e.", round(assignation_freq_t1[k,v]*100/totals_t1[k][0],2),"%")
print("\n------------------Sobre 5%-------------------")
for k,v in OrderedDict(sorted(assignation_freq_t1.items())):
    if(round(assignation_freq_t1[k,v]*100/totals_t1[k][0],2)>=5):
        print("Expected ",k,", got", v, ": ", assignation_freq_t1[k,v], "out of", totals_t1[k][0], "i.e.", round(assignation_freq_t1[k,v]*100/totals_t1[k][0],2),"%")

        

****************Modelo 1: 1 hidden layer de 90 neuronas****************
Success:  8498  out of  10000 , i.e.  84.98

!!!!!!!!!!!!!!!FREQUENCIES!!!!!!!!!!!!!!!
Expected  Ankle boot , got Ankle boot :  936 out of 1000 i.e. 93.6 %
Expected  Ankle boot , got Bag :  1 out of 1000 i.e. 0.1 %
Expected  Ankle boot , got Sandal :  24 out of 1000 i.e. 2.4 %
Expected  Ankle boot , got Sneaker :  39 out of 1000 i.e. 3.9 %
Expected  Bag , got Ankle boot :  1 out of 1000 i.e. 0.1 %
Expected  Bag , got Bag :  942 out of 1000 i.e. 94.2 %
Expected  Bag , got Coat :  7 out of 1000 i.e. 0.7 %
Expected  Bag , got Dress :  5 out of 1000 i.e. 0.5 %
Expected  Bag , got Pullover :  7 out of 1000 i.e. 0.7 %
Expected  Bag , got Sandal :  7 out of 1000 i.e. 0.7 %
Expected  Bag , got Shirt :  19 out of 1000 i.e. 1.9 %
Expected  Bag , got Sneaker :  3 out of 1000 i.e. 0.3 %
Expected  Bag , got T-shirt/top :  8 out of 1000 i.e. 0.8 %
Expected  Bag , got Trouser :  1 out of 1000 i.e. 0.1 %
Expected  Coat , got Bag :

# 3. Conclusiones y recomendaciones
Dado que para el primer modelo se obtienen accuracies sobre el 80%, y estas son bastante cercanas entre sí, este es un modelo válido. También se puede observar que los errores de clasificación son consistentes en los tres datasets, tendiendo a confundir especialmente "Shirts" con otros artículos, confundir "Pullover," "Coat" y "Shirt" entre sí, y teniendo mayor facilidad para identificar "Trouser," "Sandal" y "Bag". Por lo tanto, se considera que este es un buen modelo.

Se notó que, al aumentar en casi 1.5x la cantidad de neuronas, el modelo quedaba bastante overfitteado. Sin embargo, para este modelo también se inicializaron thetas más pequeños (de lo contrario, la función de minimize se detenía por la iteración 16), así que esto pudo haber influenciado ligeramente. Se recomienda, por lo tanto, utilizar el mismo rango y ajustes para generar los thetas iniciales al azar.

También, según los dos modelos realizados, se considerea apropiado realizar un modelo que tenga entre 90 y 133 neuronas, ya que si bien los resultados son bastante buenos, podrían mejorarse, siempre evitando caer en casos de overfitting. Esto podría ayudar a refinar las clasificaciones de las categorías que consistentemente fueron las más bajas, especialmente "Shirt," al igual que reducir en general las clasificaciones erróneas, ya que cada artículo se clasificó erróneamente como mínimo con otros 3 artículos.

A pesar de que no era requisito de implementación para el laboratorio, se recomienda también evaluar el modelo con regularización, ya que esto sería más certero y fácil de identificar los cambios necesarios al modelo, a diferencia de ir variando el número de neuronas en la capa oculta a puro tanteo (también, dado el tiempo de espera de cada corrida de un nuevo modelo, este último puede tornarse en algo muy largo).